In [17]:
import numpy as np

def first_x_sec(seconds, filepath, timecol):
    ''' param: seconds, int, filter out the first seconds of the game
        param: filepath, string, the filepath of the csv datafile
        param: timecol, int, the index of the column that contains time
    '''
    f = open(filepath)
    lines = f.readlines()
    lines = [ line.split(',') for line in lines ]
    if seconds is None:
        return np.array(lines[1:])
    else:
        return np.array(list(filter(lambda line: int(line[timecol])<=seconds, lines[1:])))

# First 5 min: 5*60=300, col 2 is time first line is header
#ability_upgrade = first_x_sec(300, 'dataset/ability_upgrades.csv', 2)
#objectives = first_x_sec(300, 'dataset/objectives.csv', 7)
player_time = first_x_sec(300, 'dataset/player_time.csv', 1)


## Match file contains more than just the label
# column 10 contains the True/False value of Radiant win
matches = first_x_sec(None, 'dataset/match.csv', None)
purchases = first_x_sec(300, 'dataset/purchase_log.csv', 1)
print(purchases)

[['44' '-81' '0' '0\n']
 ['29' '-63' '0' '0\n']
 ['43' '6' '0' '0\n']
 ..., 
 ['77' '150' '132' '49999\n']
 ['20' '150' '132' '49999\n']
 ['25' '284' '132' '49999\n']]


In [26]:
import math
print(purchases.shape)
print(purchases[1])
purchases = purchases.astype(np.int)

def set_forth(log):
        log[1] = math.ceil(max(0, log[1]) / 60) * 60
        return log

np.array([ set_forth(log) for log in purchases ])    
print(purchases[:,1])

(4014805, 4)
[ 29 -63   0   0]
[  0   0   0 ..., 120 120 240]


In [20]:
print('sdsaafgsa')

sdsaafgsa


In [4]:
labels = np.array([int(match[9] == 'True') for match in matches])
print('All the labels')
print(labels)

print('Shape of player time matrix')
print('Contains all attributes of matches, the first minutes')
print(player_time.shape)
#print(player_time[1:10])
player_time = player_time.astype(np.int)


## Separate data matches. Each index in data is one match. One match is a 32*n.
## n is the number of logs. We have data every 60 secods, so for 5 min n=5
data = [np.empty(shape=(32,0), dtype=np.int)]*50000
for x in player_time:
    data[x[0]] = np.concatenate((data[x[0]], x.reshape(32,1)), axis=1)
#Print the first match log 
print('The pre prossessed data, just the second match')
print(data[1])


All the labels
[1 0 0 ..., 1 1 0]
Shape of player time matrix
Contains all attributes of matches, the first minutes
(299994, 32)
The pre prossessed data, just the second match
[[   1    1    1    1    1    1]
 [   0   60  120  180  240  300]
 [   0   99  199  299  399  798]
 [   0    0    0    0    0    0]
 [   0   31  288  461  607  910]
 [   0  138  358  744 1146 1479]
 [   0    1    4   11   18   20]
 [   0   93  350  647  939 1348]
 [   0   99  280  441 1018 1118]
 [   0    0    3    8    8    8]
 [   0    0  144  264  493  627]
 [   0  178  326  426  651  950]
 [   0    2    3    3    6   10]
 [   0  124  165  206  495  735]
 [   0  280  542  891 1310 1574]
 [   0    2    6   12   17   21]
 [   0  327  781 1225 1682 2059]
 [   0  599  804 1314 1861 2604]
 [   0    0    3    6   11   16]
 [   0  100  222  426  942 1512]
 [   0  183  325  623  770  911]
 [   0    2    3    8    9   10]
 [   0  175  299  670  979 1253]
 [   0   99  199  299  399  582]
 [   0    0    0    0    0    1]

In [5]:
#Filter out the end state from data. 
# This will be a matrix with the game state afer x seconds in each column
# the first row is time
# following by gold, last hits and xp for each player
end_5_min_games = np.array([game[2:,-1] for game in data]).T
#print the data to look at it
print('Example data table. First 10 matches. Game state at the end of 5 min')
print(end_5_min_games[it:,:10])
# the shape should be 31, 50000. 
print('The shape of the match data table, first row is the time column, which is always 300')
print(end_5_min_games.shape)

Example data table. First 10 matches. Game state at the end of 5 min
[[1056  798 1140 1007 1739  876  902 1063 1797  836]
 [   1    0   12    4   28    8    3   16   27    8]
 [ 649  910 1291 1034 1313 1122  849  834 1504 1105]
 [1451 1479 1518 1140 1888  593 1593 1763 1168  659]
 [  13   20   14    9   16    1   22   27    8    4]
 [1810 1348 1795  700 2139  552 1323 1872 1190  656]
 [ 630 1118 1103  957 1013 1547 1043 1241 1268 1664]
 [   0    8    6    9   11   23   13    4    6   19]
 [ 504  627 1106 1347  720 1316 1474 1211 1056 1065]
 [1102  950 1163 1051  943 2087 1259  649 1318  770]
 [  15   10   14    4    1   16   13    0   15    3]
 [1888  735 1372  861  900 2031 1127  257 1960 1127]
 [1708 1574  544  893 1303  670  819 1696  760 1200]
 [  17   21    2    8   20    5    3   23    0   17]
 [1633 2059  760 1795 1265  320  746 2091  729 1867]
 [1300 2604 1209  795  677 1399 1152 1503  688 1209]
 [  11   16   12    5    0   21    0   23    0   12]
 [1901 1512 1029  955  485 128

In [4]:
from keras.layers import Dense, Activation
from keras.models import Sequential
import keras

model = Sequential()

model.add(Dense(units=50, input_dim=30))
model.add(Activation('relu'))
model.add(Dense(units=2))
model.add(Activation('softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
             optimizer=keras.optimizers.SGD(lr=0.0001, momentum=0.0, nesterov=False),
             metrics=[keras.metrics.mae, keras.metrics.categorical_accuracy])


Using TensorFlow backend.


In [5]:
print('Shapes of labels')
print(labels.shape)
from keras.utils.np_utils import to_categorical
labels_binary = to_categorical(labels)
print('example of how the labels look like')
print(labels[:10])
print('Re-structure the labels so that it is two dimensions, with prob value of each team winning')
print(labels_binary.shape)
print('example of how the binary labels look like')
print(labels_binary[:10])

# separate into train and test
x_train = end_5_min_games[:,:40000]
y_train = labels_binary[:40000,:]
x_test = end_5_min_games[:,40000:]
y_test = labels_binary[40000:,:]
print(y_test.shape)



Shapes of labels
(50000,)
example of how the labels look like
[1 0 0 0 1 1 1 1 0 0]
Re-structure the labels so that it is two dimensions, with prob value of each team winning
(50000, 2)
example of how the binary labels look like
[[ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]]
(10000, 2)


In [6]:
res = model.fit(x_train.T, y_train, epochs=50, batch_size=1000, validation_split=0.8)

Train on 7999 samples, validate on 32001 samples
Epoch 1/50
7999/7999 [==============================] - 0s - loss: 7.7593 - mean_absolute_error: 0.4818 - categorical_accuracy: 0.5182 - val_loss: 7.8044 - val_mean_absolute_error: 0.4844 - val_categorical_accuracy: 0.5156
Epoch 2/50
7999/7999 [==============================] - 0s - loss: 7.7535 - mean_absolute_error: 0.4813 - categorical_accuracy: 0.5187 - val_loss: 7.8064 - val_mean_absolute_error: 0.4846 - val_categorical_accuracy: 0.5154
Epoch 3/50
7999/7999 [==============================] - 0s - loss: 7.7613 - mean_absolute_error: 0.4817 - categorical_accuracy: 0.5183 - val_loss: 7.8074 - val_mean_absolute_error: 0.4846 - val_categorical_accuracy: 0.5155
Epoch 4/50
7999/7999 [==============================] - 0s - loss: 7.7604 - mean_absolute_error: 0.4818 - categorical_accuracy: 0.5182 - val_loss: 

In [7]:
# Test loss and accuracy
loss_and_metrics = model.evaluate(x_test.T, y_test)
print(loss_and_metrics)

 7488/10000 [=====================>........] - ETA: 0s[7.6399250358581545, 0.47409038228988648, 0.52590000000000003]
